Import libraries

In [40]:
import pandas as pd
import os
import re
import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="white")
pd.set_option('max_rows', 10)
pd.plotting.register_matplotlib_converters()
%matplotlib inline

Parse in the banzuke (rankings) by decade. 

In [41]:
#create the ranking dataframes by decade
ranking_1950 = pd.DataFrame([],columns=("rank_code", "name", "birthplace", "stable", "dob", "height", "weight", "rank_no", "rank_date", "division", "age")) 
ranking_1960 = pd.DataFrame([],columns=("rank_code", "name", "birthplace", "stable", "dob", "height", "weight", "rank_no", "rank_date", "division", "age")) 
ranking_1970 = pd.DataFrame([],columns=("rank_code", "name", "birthplace", "stable", "dob", "height", "weight", "rank_no", "rank_date", "division", "age")) 
ranking_1980 = pd.DataFrame([],columns=("rank_code", "name", "birthplace", "stable", "dob", "height", "weight", "rank_no", "rank_date", "division", "age")) 
ranking_1990 = pd.DataFrame([],columns=("rank_code", "name", "birthplace", "stable", "dob", "height", "weight", "rank_no", "rank_date", "division", "age")) 
ranking_2000 = pd.DataFrame([],columns=("rank_code", "name", "birthplace", "stable", "dob", "height", "weight", "rank_no", "rank_date", "division", "age")) 
ranking_2010 = pd.DataFrame([],columns=("rank_code", "name", "birthplace", "stable", "dob", "height", "weight", "rank_no", "rank_date", "division", "age")) 
ranking_2020 = pd.DataFrame([],columns=("rank_code", "name", "birthplace", "stable", "dob", "height", "weight", "rank_no", "rank_date", "division", "age")) 

def parse_ranking_data(path, df):

    #set the path to the ranking files
    ranking_file_path = os.chdir(path)

    #loop through each file in the decade folder
    for files in os.listdir(ranking_file_path):           
                    
        file_name = files
        files = open(file_name, "r")
        lines = files.readlines()
        files.close()

        #reset rank after each file is read
        rank_no = 0

        #get the date of the sumo ranking
        rank_date_string = file_name[0:4] + "-" + file_name[5:7]
        rank_date = datetime.datetime.strptime(rank_date_string, "%Y-%m")

        #parse in the data by looping through each line in the file
        for line in lines:

            #use the number of spaces in the line and the beginning of the rank codes to filter out the irrelevant lines                    
            if "    " in line and "changed from" not in line and line.startswith("Bg") == False and line.startswith("Sj") == False and line.startswith("Mz") == False:
                line = line + " "

                #remove extra spaces
                processed_line = re.sub(' +', ' ', line)

                #separates the columns 
                split = processed_line.split(" ")
                
                #splits the rank from the name if the longer names merged
                if len(split[0]) >= 7:
                    split[6] = split[5]
                    split[5] = split[4]
                    split[4] = split[3]
                    split[3] = split[2]
                    split[2] = split[1]
                    split[1] = split[0][7:]
                    split[0] = split[0][:7]
                    
                #splits the name from the birthplace if the strings merged
                if len(split[1]) >= 15:
                    charCount = 0
                    upperCount = 0

                    for char in split[1]:
                        charCount +=1

                        if char == char.upper():
                            upperCount += 1

                            if upperCount == 2:
                                split[6] = split[5]
                                split[5] = split[4]
                                split[4] = split[3]
                                split[3] = split[2]
                                split[2] = split[1][charCount:]
                                split[1] = split[1][:charCount] 

                #splits the birthplace and the stable if they have merged
                if len(split[2]) >= 12:
                    charCount = 0
                    upperCount = 0

                    for char in split[2]:
                        charCount +=1

                        if char == char.upper():
                            upperCount += 1

                            if upperCount == 2:
                                split[6] = split[5]
                                split[5] = split[4]
                                split[4] = split[3]
                                split[3] = split[2][charCount-1:]
                                split[2] = split[2][:charCount-1]

                #corrects the split if birthplace is Hong Kong or Sri Lanka
                if split[2] == "Hong" and split[3][:4] == "Kong":
                    split[3] = split[4]
                    split[4] = split[5]
                    split[5] = split[6]
                    split[6] = split[7]                      
                    split[2] = "Hong Kong" 

                elif split[2] == "Sri" and split[3][:4] == "Lank":
                    split[3] = split[4]
                    split[4] = split[5]
                    split[5] = split[6]
                    split[6] = split[7]                      
                    split[2] = "Sri Lanka" 

                #splits the birthplace from the stable if the birthplace has a space
                if len(split[3]) >= 6:
                    charCount = 0
                    upperCount = 0

                    for char in split[3]:
                        charCount +=1

                        if char == char.upper():
                            upperCount += 1

                            if upperCount == 2: 
                                split[2] = split[2] + " " + split[3][:charCount-1].title()
                                split[3] = split[3][charCount-1:]

                #default column layout
                rank_code = split[0]
                name = split[1]
                birthplace = split[2]
                stable = split[3]
                dob = split[4]
                height = split[5]
                weight = split[6]
                rank_no = rank_no + 1

                #convert height to int
                if height[0] == "-":
                    height = "-"
                else: 
                    height = float(height)

                #convert height to int
                if weight[0] == "-":
                    weight = "-"
                else: 
                    weight = float(weight)

                #convert dob to date
                if dob[0] == "-":
                    dob = "-"
                else:
                    dob = dob[6:11] + "-" + dob[3:5] + "-" +  dob[0:2]
                    dob = datetime.datetime.strptime(dob, "%Y-%m-%d")

                #calculate age if dob is given
                if dob == "-":
                    age = "-"
                else:
                    age = rank_date - dob
                
                #set division
                if rank_code[0].upper() == "Y" or rank_code[0].upper() == "O" or rank_code[0].upper() == "K":
                    division = "Makuuchi"
                elif rank_code[0].upper() == "S":
                    if rank_code[1].upper() == "D":
                        division = "Sandanme"
                    else:
                        division = "Makuuchi"
                elif rank_code[0].upper() == "M":
                    if rank_code[1].upper() == "S":
                        division = "Makushita"
                    else:
                        division = "Makuuchi"
                elif rank_code[0].upper() == "J":
                    if rank_code[1].upper() == "D":
                        division = "Jonidan"
                    elif rank_code[1].upper() == "K":
                        division = "Jonokuchi"
                    else:
                        division = "Juryo"

                #apply the data to the dataframe
                df.loc[len(df.index)] = [rank_code, name, birthplace, stable, dob, height, weight, rank_no, rank_date, division, age]
                
#parse the data for each decade
parse_ranking_data("C:/Users/Josh/Documents/Uni/Dissertation/Data/sumo/banzuke/1950/", ranking_1950)
parse_ranking_data("C:/Users/Josh/Documents/Uni/Dissertation/Data/sumo/banzuke/1960/", ranking_1960)
parse_ranking_data("C:/Users/Josh/Documents/Uni/Dissertation/Data/sumo/banzuke/1970/", ranking_1970)
parse_ranking_data("C:/Users/Josh/Documents/Uni/Dissertation/Data/sumo/banzuke/1980/", ranking_1980)
parse_ranking_data("C:/Users/Josh/Documents/Uni/Dissertation/Data/sumo/banzuke/1990/", ranking_1990)
parse_ranking_data("C:/Users/Josh/Documents/Uni/Dissertation/Data/sumo/banzuke/2000/", ranking_2000)
parse_ranking_data("C:/Users/Josh/Documents/Uni/Dissertation/Data/sumo/banzuke/2010/", ranking_2010)
parse_ranking_data("C:/Users/Josh/Documents/Uni/Dissertation/Data/sumo/banzuke/2020/", ranking_2020)

Delete the "day 16" files from the tournament folder as they do not provide bout insights. Then, parse in each tournament file by decade.

In [42]:
#create the tournament dataframes by decade
tournament_1950 = pd.DataFrame(columns=("date", "day_no", "division", "win_technique","rank_1", "name_1", "record_1", "rank_2", "name_2", "record_2"))
tournament_1960 = pd.DataFrame(columns=("date", "day_no", "division", "win_technique","rank_1", "name_1", "record_1", "rank_2", "name_2", "record_2"))
tournament_1970 = pd.DataFrame(columns=("date", "day_no", "division", "win_technique","rank_1", "name_1", "record_1", "rank_2", "name_2", "record_2"))
tournament_1980 = pd.DataFrame(columns=("date", "day_no", "division", "win_technique","rank_1", "name_1", "record_1", "rank_2", "name_2", "record_2"))
tournament_1990 = pd.DataFrame(columns=("date", "day_no", "division", "win_technique","rank_1", "name_1", "record_1", "rank_2", "name_2", "record_2"))
tournament_2000 = pd.DataFrame(columns=("date", "day_no", "division", "win_technique","rank_1", "name_1", "record_1", "rank_2", "name_2", "record_2"))
tournament_2010 = pd.DataFrame(columns=("date", "day_no", "division", "win_technique","rank_1", "name_1", "record_1", "rank_2", "name_2", "record_2"))
tournament_2020 = pd.DataFrame(columns=("date", "day_no", "division", "win_technique","rank_1", "name_1", "record_1", "rank_2", "name_2", "record_2"))

def parse_tournament_data(path, df):    
    #set the path to the tournament file
    tournament_file_path = os.chdir(path)

    #loop through each file in the decade folder
    for files in os.listdir(tournament_file_path):    

        file_name = files
        files = open(file_name, "r")
        lines = files.readlines()
        files.close()

        #get date of the tournament
        tournament_date_string = file_name[0:4] + "-" + file_name[5:7]
        tournament_date = datetime.datetime.strptime(tournament_date_string, "%Y-%m")

        #get the day of the tournament
        tournament_day_no = int(file_name[-6:-4])

        #parse in the data by looping through each line in the file
        for line in lines:

            #get the division of the bout
            if "Makuuchi" in line:
                division = "Makuuchi"
            elif "Juryo" in line:
                division = "Juryo"
            elif "Makushita" in line:
                division = "Makushita"
            elif "Sandanme" in line:
                division = "Sandanme"
            elif "Jonidan" in line:
                division = "Jonidan"
            elif "Jonokuchi" in line:
                division = "Jonokuchi"

            #use the number of spaces in the line and the beginning of the rank codes to filter out the irrelevant lines                     
            if "    " in line and "changed from" not in line and line.startswith("Bg") == False and line.startswith("Sj") == False and line.startswith("Mz") == False:
                line = line + " "

                #remove extra spaces
                processed_line = re.sub(' +', ' ', line)

                #separates the columns 
                split = processed_line.split(" ")

                #adjust the splits if winning_technique merges with wrestler_2_rank
                if len(split[3]) >= 16:
                    split[6] = split[5]
                    split[5] = split[4]
                    split[4] = split[3][16:]
                    split[3] = split[3][:16]
                    split[2] = split[2]
                    split[1] = split[1]
                    split[0] = split[0]

                #default column layout
                wrestler_1_rank = split[0]
                wrestler_1_name = split[1]
                wrestler_1_record = split[2]
                winning_technique = split[3]
                wrestler_2_rank = split[4]
                wrestler_2_name = split[5]
                wrestler_2_record = split[6]

                #apply the parsed data to the dataframe
                df.loc[len(df.index)] = [tournament_date, tournament_day_no, division, winning_technique, wrestler_1_rank, 
                                        wrestler_1_name, wrestler_1_record, wrestler_2_rank, wrestler_2_name, wrestler_2_record]

#parse the data for each decade
parse_tournament_data("C:/Users/Josh/Documents/Uni/Dissertation/Data/sumo/torikumi/1950/", tournament_1950)
parse_tournament_data("C:/Users/Josh/Documents/Uni/Dissertation/Data/sumo/torikumi/1960/", tournament_1960)
parse_tournament_data("C:/Users/Josh/Documents/Uni/Dissertation/Data/sumo/torikumi/1970/", tournament_1970)
parse_tournament_data("C:/Users/Josh/Documents/Uni/Dissertation/Data/sumo/torikumi/1980/", tournament_1980)
parse_tournament_data("C:/Users/Josh/Documents/Uni/Dissertation/Data/sumo/torikumi/1990/", tournament_1990)
parse_tournament_data("C:/Users/Josh/Documents/Uni/Dissertation/Data/sumo/torikumi/2000/", tournament_2000)
parse_tournament_data("C:/Users/Josh/Documents/Uni/Dissertation/Data/sumo/torikumi/2010/", tournament_2010)
parse_tournament_data("C:/Users/Josh/Documents/Uni/Dissertation/Data/sumo/torikumi/2020/", tournament_2020)

Parse in the name changes from the banzuke files.

In [43]:
changed_names_1950 = pd.DataFrame(columns=("rank", "old_name", "new_name", "date_changed")) 
changed_names_1960 = pd.DataFrame(columns=("rank", "old_name", "new_name", "date_changed")) 
changed_names_1970 = pd.DataFrame(columns=("rank", "old_name", "new_name", "date_changed")) 
changed_names_1980 = pd.DataFrame(columns=("rank", "old_name", "new_name", "date_changed")) 
changed_names_1990 = pd.DataFrame(columns=("rank", "old_name", "new_name", "date_changed")) 
changed_names_2000 = pd.DataFrame(columns=("rank", "old_name", "new_name", "date_changed")) 
changed_names_2010 = pd.DataFrame(columns=("rank", "old_name", "new_name", "date_changed")) 
changed_names_2020 = pd.DataFrame(columns=("rank", "old_name", "new_name", "date_changed")) 

def parse_name_changes(path, df):

    #set directory
    name_changes_path = os.chdir(path)
    #run through all files in directory
    for files in os.listdir(name_changes_path):           
            
        file_name = files
        files = open(file_name, "r")
        lines = files.readlines()
        files.close()

        #get date of the name change
        rank_date_string = file_name[0:4] + "-" + file_name[5:7]
        rank_date = datetime.datetime.strptime(rank_date_string, "%Y-%m")

        #get the sumo data
        for line in lines:
                                
            if "changed from" in line:
                
                line = line + " "

                #remove extra spaces
                processed_line = re.sub(' +', ' ', line)

                #separates the columns 
                split = processed_line.split(" ")
                
                rank = split[0]
                new_name = split[1]
                old_name = split[4]

                df.loc[len(df.index)] = [rank, old_name, new_name, rank_date]

#parse the data for each decade
parse_name_changes("C:/Users/Josh/Documents/Uni/Dissertation/Data/sumo/banzuke/1950/",changed_names_1950)
parse_name_changes("C:/Users/Josh/Documents/Uni/Dissertation/Data/sumo/banzuke/1960/",changed_names_1960)
parse_name_changes("C:/Users/Josh/Documents/Uni/Dissertation/Data/sumo/banzuke/1970/",changed_names_1970)
parse_name_changes("C:/Users/Josh/Documents/Uni/Dissertation/Data/sumo/banzuke/1980/",changed_names_1980)
parse_name_changes("C:/Users/Josh/Documents/Uni/Dissertation/Data/sumo/banzuke/1990/",changed_names_1990)
parse_name_changes("C:/Users/Josh/Documents/Uni/Dissertation/Data/sumo/banzuke/2000/",changed_names_2000)
parse_name_changes("C:/Users/Josh/Documents/Uni/Dissertation/Data/sumo/banzuke/2010/",changed_names_2010)
parse_name_changes("C:/Users/Josh/Documents/Uni/Dissertation/Data/sumo/banzuke/2020/",changed_names_2020)

#concat the name changes by decade into one dataframe
changed_names = pd.concat([changed_names_1950, changed_names_1960, changed_names_1970, changed_names_1980, 
                    changed_names_1990, changed_names_2000, changed_names_2010, changed_names_2020], 
                    ignore_index = True, sort = False)

Cleaning Stage:
Remove ranking data outside of the top two divisions prior to 1990 as there is only tournament data for the top two division data prior to 1988, and then 1988-1989 the lower divisions have many missing values. After 1990 all data is available.

In [44]:
#keeps only the rows for wrestlers in the top two divisions between 1950 and 1979
ranking_1950 = ranking_1950[ranking_1950["division"].isin(["Makuuchi", "Juryo"]) == True]
ranking_1960 = ranking_1960[ranking_1960["division"].isin(["Makuuchi", "Juryo"]) == True]
ranking_1970 = ranking_1970[ranking_1970["division"].isin(["Makuuchi", "Juryo"]) == True]
ranking_1980 = ranking_1980[ranking_1980["division"].isin(["Makuuchi", "Juryo"]) == True]

#reset the index after removing the rows
ranking_1950.reset_index(drop=True, inplace=True)
ranking_1960.reset_index(drop=True, inplace=True)
ranking_1970.reset_index(drop=True, inplace=True)
ranking_1980.reset_index(drop=True, inplace=True)

Append each decade of ranking and tournament dataframes back together and reset the index.

In [45]:
#appending the ranking dataframes
ranking_df = ranking_1950.append(ranking_1960).append(ranking_1970).append(ranking_1980)
ranking_df = ranking_df.append(ranking_1990).append(ranking_2000).append(ranking_2010).append(ranking_2020)

#appending the tournament dataframes
tournament_df = tournament_1950.append(tournament_1960).append(tournament_1970).append(tournament_1980)
tournament_df = tournament_df.append(tournament_1990).append(tournament_2000).append(tournament_2010).append(tournament_2020)

#reset the indexes of the two dataframes
ranking_df.reset_index(drop=True, inplace=True)
tournament_df.reset_index(drop=True, inplace=True)

Set NaNs for all cells without values

In [46]:
#convert dashes to NaN for ranking dateframes
ranking_df = ranking_df.replace("-", np.NaN)

#convert dashes to NaN for tournament dateframes
tournament_df = tournament_df.replace("-", np.NaN)

Remove the hashtages from the ranking data that were indicating name changes.

In [47]:
#delete hashtags within the "name" column
ranking_df["name"] = ranking_df["name"].str.replace("#","")

Convert all date and time columns to just date

In [48]:
ranking_df["dob"] = pd.to_datetime(ranking_df["dob"]).dt.normalize()
ranking_df["rank_date"] = pd.to_datetime(ranking_df["rank_date"]).dt.normalize()
tournament_df["date"] = pd.to_datetime(tournament_df["date"]).dt.normalize()

Change the wrestler's age in days to years for the ranking dataframes.

In [49]:
#convert age column to string
ranking_df["age"] = ranking_df["age"].astype("string")

#remove the "days"
ranking_df["age"] = ranking_df["age"].str.replace(" days","")

#convert to int
ranking_df["age"] = ranking_df["age"].astype("int")

#set NaN ages to zero, then divide the years, then convert zero values back to NaN
ranking_df["age"] = ((ranking_df["age"].fillna(0)) / 365.25).replace(0, np.NaN)

Remove brackets, "d" and "\n" from the wrestler records in all the tournament dataframes.

In [50]:
#removing brackets, "d" and "\n"
tournament_df["record_1"] = tournament_df["record_1"].str.replace("(","")
tournament_df["record_2"] = tournament_df["record_2"].str.replace("(","")
tournament_df["record_1"] = tournament_df["record_1"].str.replace(")","")
tournament_df["record_2"] = tournament_df["record_2"].str.replace(")","")
tournament_df["record_1"] = tournament_df["record_1"].str.replace("d","")
tournament_df["record_2"] = tournament_df["record_2"].str.replace("d","")
tournament_df["record_2"] = tournament_df["record_2"].str.replace("\n","")

C:\Users\Josh\AppData\Local\Temp/ipykernel_4964/2534270476.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  tournament_df["record_1"] = tournament_df["record_1"].str.replace("(","")
C:\Users\Josh\AppData\Local\Temp/ipykernel_4964/2534270476.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  tournament_df["record_2"] = tournament_df["record_2"].str.replace("(","")
C:\Users\Josh\AppData\Local\Temp/ipykernel_4964/2534270476.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  tournament_df["record_1"] = tournament_df["re

Change the format of the wrestler's records so that all are win-loss-missed 

In [51]:
#add a second dash to both wrestler record columns
def format_wrestler_records(column):
    
    for i, row in tournament_df.iterrows():
        dashes = row[column].count("-")
        
        if dashes == 1:
            tournament_df.at[i, column] = tournament_df.at[i, column] + "-0"

format_wrestler_records("record_1")
format_wrestler_records("record_2")

Split the record columns into three columns - one for wins, one for losses and one for missed bouts

In [52]:
#split on dashes for each wrestler using their record column to create three new columns
tournament_df[["wins_1", "losses_1", "missed_1"]] = tournament_df.record_1.str.split("-", expand=True)
tournament_df[["wins_2", "losses_2", "missed_2"]] = tournament_df.record_2.str.split("-", expand=True)

Remove the original record column now that the wins, losses, missed columns have been created 

In [53]:
#drop "record_1" and "record_2" columns for each data frame
tournament_df = tournament_df.drop("record_1", axis=1)
tournament_df = tournament_df.drop("record_2", axis=1)

Remove hashtags, commas and "\n" from the names in the changed names dataframe

In [54]:
#formatting the changed names to remove unnecessary characters
changed_names["old_name"] = changed_names["old_name"].str.replace("#","")
changed_names["new_name"] = changed_names["new_name"].str.replace("#","")
changed_names["old_name"] = changed_names["old_name"].str.replace(",","")
changed_names["new_name"] = changed_names["new_name"].str.replace(",","")
changed_names["old_name"] = changed_names["old_name"].str.replace("\n","")
changed_names["new_name"] = changed_names["new_name"].str.replace("\n","")


Add the name changes to the rankings data frames

In [55]:
#find rows with name changes by using the name, rank and date of the change
rows_with_name_changes = ranking_df[ranking_df.set_index(['name','rank_code', 'rank_date']).index.isin(changed_names.set_index(['new_name','rank', 'date_changed']).index)]
    
#convert the rows with name changes to an array of row numbers
index_of_rows_with_name_changes = rows_with_name_changes.index.values

#add a new column for name changes 
ranking_df.loc[index_of_rows_with_name_changes, "other_name(s)"] = "yes" #need to set this to the changed name as "changed_names.old_name" is setting the wrong name (index)



Create unique identifiers on the ranking dataframe by using the birth place and the birth date

In [56]:
ranking_df['wrestler_id'] = ranking_df.dob.astype(str) + '_' + ranking_df.birthplace.astype(str)

Save the dataframes to Excel files

In [57]:
#set the paths for the cleaned dataframes to be saved
cleaned_data_path = os.chdir('C:/Users/Josh/Documents/Uni/Dissertation/Data/sumo/cleaned_datasets/')

#save the cleaned dataframes as excel documents
ranking_df.to_excel('cleaned_ranking.xlsx', sheet_name='sheet1', index=False)
tournament_df.to_excel('cleaned_tournament.xlsx', sheet_name='sheet1', index=False)
changed_names.to_excel('cleaned_changed_names.xlsx', sheet_name='sheet1', index=False)

*temp - load the most up-to-date version of the dataframes*

In [58]:
#load the cleaned ranking data
ranking_df = pd.read_excel('C:/Users/Josh/Documents/Uni/Dissertation/Data/sumo/cleaned_datasets/cleaned_ranking.xlsx')

#load the cleaned tournament data
tournament_df = pd.read_excel("C:/Users/Josh/Documents/Uni/Dissertation/Data/sumo/cleaned_datasets/cleaned_tournament.xlsx")

#load the cleaned name changes
changed_names = pd.read_excel("C:/Users/Josh/Documents/Uni/Dissertation/Data/sumo/cleaned_datasets/cleaned_changed_names.xlsx")